In [1]:
import QuantLib as ql

In [2]:
valuationDate = ql.Date(24, 11, 2020) # 평가일자
ql.Settings.instance().evaluationDate = valuationDate # 퀀트립에 평가하는 날짜를 알려주기 위함

calendar = ql.SouthKorea() # 한국 달력
dayCount = ql.ActualActual() # 이자 계산 방식

# Simple Quote Objects(시장 변수)
underlying_qt = ql.SimpleQuote(270.48)      # 현재 주가
dividend_qt = ql.SimpleQuote(0.0)           # 배당(0은 배당 없음을 의미)
riskfreerate_qt = ql.SimpleQuote(0.01)      # 금리(0.01 -> 1%)
volatility_qt = ql.SimpleQuote(0.13)        # 변동성(0.13 -> 13%)

# Quote Handle Objects(Quote 변수들을 특정 지점의 값이 아닌 쿼트 핸들 형태로 만들어야 함)
u_qhd = ql.QuoteHandle(underlying_qt)
d_qhd = ql.QuoteHandle(dividend_qt)
r_qhd = ql.QuoteHandle(riskfreerate_qt)
v_qhd = ql.QuoteHandle(volatility_qt)

# Term-Structure Objects(Quote 변수들을 쿼트 핸들의 형태에서 기간 형태로 만들어야 함)
r_ts = ql.FlatForward(valuationDate, r_qhd, dayCount) # FlatForward ==> Quote의 값이 전 만기에 동일한 값을 가진다는 것을 정의하는 것
d_ts = ql.FlatForward(valuationDate, d_qhd, dayCount) # (평가일자, 값, 이자 계산 방식)
v_ts = ql.BlackConstantVol(valuationDate, calendar, v_qhd, dayCount) # BlackConstantVol ==> FlatForward와 동일한 기능
                          # (평가일자, 달력, 값, 이자 계산 방식)
# Term-Structure Handle Objects
r_thd = ql.YieldTermStructureHandle(r_ts)
d_thd = ql.YieldTermStructureHandle(d_ts)
v_thd = ql.BlackVolTermStructureHandle(v_ts)

# Process & Engine
process = ql.BlackScholesMertonProcess(u_qhd, d_thd, r_thd, v_thd) # (주가, 배당, 금리, 변동성)
# BlackScholesMertonProcess ==> 기초자산의 수익률이 정규분포를 따르고, 랜덤하게 움직인다는 정의를 가진 프로세스
engine = ql.AnalyticEuropeanEngine(process) # AnalyticEuropeanEngine ==> 엔진을 구현

# Option Objects
option_type = ql.Option.Call
strikePrice = 272 # 행사가(만기 페이오프를 계산했을 때 행사가(272)가 넘으면 이득을 보게 되는 것)
expiryDate = ql.Date(24, 11, 2021) # 만기일
exercise = ql.EuropeanExercise(expiryDate) # EuropeanExercise 행사 방식을 선택
payoff = ql.PlainVanillaPayoff(option_type, strikePrice) # 페이오프 설정
option = ql.VanillaOption(payoff, exercise) # 바닐라 옵션 구현

# Pricing
option.setPricingEngine(engine)

In [3]:
# Price & Greeks Results
print('Option Premium = ', round(option.NPV(), 2))          # option premium
print('Option Delta = ', round(option.delta(), 4))          # Delta ==> 주가가 1단위 움직였을 때 옵션이 얼마나 움직이는지
print('Option Gamma = ', round(option.gamma(), 4))          # gamma
print('Option Theta = ', round(option.thetaPerDay(), 4))    # theta ==> 날짜가 하루 움직였을 때 옵션이 얼마나 움직이는지
print('Option Vega = ', round(option.vega() / 100, 4))      # vega
print('Option Rho = ', round(option.rho() / 100, 4))        # rho ==> 금리 가격 변화
print('\n')

# Automatic Re-Pricing
underlying_qt.setValue(275) # underlying_qt를 기존 272에서 275로 변환하라는 의미
print('Option Premium = ', round(option.NPV(), 2))          # option premium
print('Option Delta = ', round(option.delta(), 4))          # delta
print('Option Gamma = ', round(option.gamma(), 4))          # gamma
print('Option Theta = ', round(option.thetaPerDay(), 4))    # theta
print('Option Vega = ', round(option.vega() / 100, 4))      # vega
print('Option Rho = ', round(option.rho() / 100, 4))        # rho
print('\n')

# Implied Volatility(내재 변동성)
underlying_qt.setValue(270.48)
mkt_price = 8.21
implied_volatility = option.impliedVolatility(mkt_price, process) # 내재 변동성 매서드 이용(임의의 변동성(0.13)이 아닌 계산된 변동성)
print("Implied_volatility = ", round(implied_volatility, 2)) # 임의로 지정한 0.13이 아닌 0.07로 나옴

volatility_qt.setValue(implied_volatility)
print('Option Premium = ', round(option.NPV(), 2))          # option premium
print('Option Delta = ', round(option.delta(), 4))          # delta
print('Option Gamma = ', round(option.gamma(), 4))          # gamma
print('Option Theta = ', round(option.thetaPerDay(), 4))    # theta
print('Option Vega = ', round(option.vega() / 100, 4))      # vega
print('Option Rho = ', round(option.rho() / 100, 4))        # rho
print('\n')

Option Premium =  14.59
Option Delta =  0.5393
Option Gamma =  0.0113
Option Theta =  -0.0227
Option Vega =  1.0737
Option Rho =  1.3126


Option Premium =  17.14
Option Delta =  0.5895
Option Gamma =  0.0109
Option Theta =  -0.023
Option Vega =  1.0692
Option Rho =  1.4494


Implied_volatility =  0.07
Option Premium =  8.21
Option Delta =  0.5388
Option Gamma =  0.0208
Option Theta =  -0.0142
Option Vega =  1.0738
Option Rho =  1.375


